In [4]:
# Block 0: Documentation

print('Program to list and download ABI data files from Amazon Web Services (AWS), AMS Short Course, March 18, 2021\n')
print('Version 1.0, February 8, 2021\n')
print('Written by Dr. Amy Huff (IMSG at NOAA/NESDIS/STAR) and Ryan Theurer (GVT LLC at NOAA/NESDIS/STAR)\n')
print('For questions contact Dr. Huff: amy.huff@noaa.gov\n')
print('This program accesses the ABI data archive on AWS and lists/downloads data files for a given date and time period specified by the user.\n')
print('Block 1 imports modules and libraries, and blocks 2-3 are functions that require no input from the user; there is no visible output from these blocks. In block 4, the user enters settings and obtains output.')

Program to list and download ABI data files from Amazon Web Services (AWS), AMS Short Course, March 18, 2021

Version 1.0, February 8, 2021

Written by Dr. Amy Huff (IMSG at NOAA/NESDIS/STAR) and Ryan Theurer (GVT LLC at NOAA/NESDIS/STAR)

For questions contact Dr. Huff: amy.huff@noaa.gov

This program accesses the ABI data archive on AWS and lists/downloads data files for a given date and time period specified by the user.

Block 1 imports modules and libraries, and blocks 2-3 are functions that require no input from the user; there is no visible output from these blocks. In block 4, the user enters settings and obtains output.


In [5]:
# Block 1: Import modules and libraries

# Library to perform array operations
import numpy as np

# Module to interface with s3 (AWS)
import s3fs

# Module for manipulating dates and times
import datetime

# Module to access files in the directory
import os

In [6]:
# Block 2: Find Julian day from given year/month/day
# "year", "month", and "day" are global variables set in final block

def julian(year, month, day):
    calendar = datetime.datetime(year, month, day)
    julian_day = calendar.strftime('%j')
    
    return julian_day

In [65]:
# Block 3: Create array containing ABI data file names for given satellite/product and date/time period
# "year", "month", "day", "start", "end", "satellite", and "product" are global variables set in final block

def aws_list(year, month, day, start, end, satellite, product, channel, meso):
  
    # Access AWS using anonymous credentials
    aws = s3fs.S3FileSystem(anon=True)

    # Make a list of all data files encompassing given date and start/end hours
    julian_day = julian(year, month, day)
    #start_time = str(start).zfill(4)[0:2]
    start_time = start[0:2]
    #end_time = str(end).zfill(4)[0:2]
    end_time = end[0:2]
    hour_range = range(int(start_time), int(end_time) + 1)
    print(hour_range)
    final_list = []
    for i in hour_range:
        i_str = str(i).zfill(2)
        #hour_files = aws.ls('noaa-goes' + str(satellite) + '/' + product + '/' + str(year) + '/' + julian_day + '/' + str(i) + '/')
        hour_files = aws.ls('noaa-goes' + str(satellite) + '/' + product + '/' + str(year) + '/' + julian_day + '/' + i_str + '/')
        final_list.extend(hour_files) 
        all_hours = np.array(final_list)

    # Extract list of data files for specified period set by start/end times
    data = []
    # List file names
    for i in all_hours:
        if len(meso) > 0:
            if (i[-42:-38] >= str(start) and i[-42:-38] <= str(end)) and (i[-58:-55] == channel) and (i[-63:-61] == meso):
                data.append(i)
        elif (i[-42:-38] >= str(start) and i[-42:-38] <= str(end)) and (i[-58:-55] == channel):
            data.append(i)
        else:
            continue

    return data

In [85]:
# Block 4: Enter user settings and list ABI data files, with option to save files locally

# Satellite and product settings
satellite = 16  # GOES-East = 16, GOES-West = 17
#product = 'ABI-L2-AODC'  # ABI product name abbreviation; see list at https://docs.opendata.aws/noaa-goes16/cics-readme.html
#product = 'ABI-L1b-RadC'
product = 'ABI-L1b-RadM'

# Day and time settings
year = 2022    # 4-digit year (e.g., 2021)
month = 2    # 1- or 2-digit month (e.g., Feb = 2, Oct = 10)
#day = 1    # 1- or 2- digit day (e.g., 1, 25)
days = range(17,18)
start = '1800'    # 4-digit observation start time in UTC, no colon (e.g. 20:00 UTC = 2000)
end = '1859'    # 4-digit observation end time in UTC, no colon (e.g. 20:35 UTC = 2035)
meso = 'M2'   # String starting with 'M' for Mesosector; valid values: empty string, M1 and M2
#channels = ['C01','C08','C13']  # string list starting with 'C' for channel; valid values C01-C16
#channels = ['C01','C02','C03','C04','C05','C06','C07','C08','C09','C10',
#            'C11','C12','C13','C14','C15','C16']  # string list starting with 'C' for channel; valid values C01-C16
channels = ['C01']

# Data file saving settings
save = 'yes'  # Option to save data files: 'yes' (save to "file_path" directory) or 'no' (list file names only)
#save_path = os.getcwd() + '/data/'  # Directory where data files will be saved
#save_path = '/home/disk/bob/impacts/raw/goes16/CONUS/Channel01/'  # Directory where data files will be saved
#save_path_base = '/home/disk/bob/impacts/raw/goes16/CONUS' # Directory where data files will be saved
save_path_base = '/home/disk/bob/impacts/raw/goes16/Mesoscale-2' # Directory where data files will be saved

#########################################################################################################################

if __name__ == "__main__":

    for day in days:
        print(day)
        for channel in channels:
            print(channel)
            yyyymmdd = str(year)+str(month).zfill(2)+str(day).zfill(2)
            save_path = save_path_base+'/Channel'+channel[1:3]+'/'+yyyymmdd+'/'  # Directory where data files will be saved
            # Query AWS and list filenames matching entered settings
            data = aws_list(year, month, day, start, end, satellite, product, channel, meso)
            if len(data) > 0:
                for i in data:
                    print(i.split('/')[-1])
            else:
                print('No files retrieved.  Check settings and try again.')

            # Downlad and save data files to specfied directory
            if save == 'yes':
                if not os.path.isdir(save_path):
                    os.makedirs(save_path)
                aws = s3fs.S3FileSystem(anon=True)
                for i in data:
                    aws.get(i, save_path + i.split('/')[-1])
                print('Download complete!')
            else:
                pass

17
C01
range(18, 19)
OR_ABI-L1b-RadM2-M6C01_G16_s20220481800554_e20220481801011_c20220481801045.nc
OR_ABI-L1b-RadM2-M6C01_G16_s20220481801554_e20220481802011_c20220481802046.nc
OR_ABI-L1b-RadM2-M6C01_G16_s20220481802554_e20220481803011_c20220481803051.nc
OR_ABI-L1b-RadM2-M6C01_G16_s20220481803554_e20220481804011_c20220481804048.nc
OR_ABI-L1b-RadM2-M6C01_G16_s20220481804554_e20220481805011_c20220481805045.nc
OR_ABI-L1b-RadM2-M6C01_G16_s20220481805554_e20220481806011_c20220481806045.nc
OR_ABI-L1b-RadM2-M6C01_G16_s20220481806554_e20220481807011_c20220481807051.nc
OR_ABI-L1b-RadM2-M6C01_G16_s20220481809554_e20220481810011_c20220481810046.nc
OR_ABI-L1b-RadM2-M6C01_G16_s20220481812554_e20220481813011_c20220481813045.nc
OR_ABI-L1b-RadM2-M6C01_G16_s20220481813554_e20220481814011_c20220481814047.nc
OR_ABI-L1b-RadM2-M6C01_G16_s20220481814554_e20220481815011_c20220481815045.nc
OR_ABI-L1b-RadM2-M6C01_G16_s20220481815554_e20220481816011_c20220481816048.nc
OR_ABI-L1b-RadM2-M6C01_G16_s20220481816554_